In [4]:
import os
from langchain.chains import LLMChain
from dotenv import load_dotenv, find_dotenv
from langchain import HuggingFaceHub

load_dotenv(find_dotenv())

True

In [5]:
llm = HuggingFaceHub(repo_id="tiiuae/falcon-7b-instruct", model_kwargs={"max_length":64, "max_new_tokens":100})


/Users/atorres/.local/share/virtualenvs/langchain-kv9OLnAB-python/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## PromptTemplates 
Prompt Templates makes creating a prompt for different usecases easier than using f-strings and interpolate the videos

In [6]:
template = """
Interprete the text and evaluate the text.
sentiment: is the text in a positive, neutral or negative sentiment?
subject: What subject is the text about? Use exactly one word.

Format the output as JSON with the following keys:
sentiment
subject

text: {input}
"""

In [7]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template=template)
chain = LLMChain(llm=llm, prompt=prompt_template)
chain.predict(input="I ordered Pizza Salami and it was awesome!")

'\nThe sentiment of the text is positive. The subject of the text is Pizza Salami.'

## Real World example with ResponseSchema, Templates, Chains and OutputParsers
There were two issues with the output: The output also contains text and the output is just a string which can not just the converted to a dictionary. Lets make it better with a more complex example

In [8]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

sentiment_schema = ResponseSchema(
    name="sentiment",
    description="Is the text positive, neutral or negative? Only provide these words",
)
subject_schema = ResponseSchema(
    name="subject", description="What subject is the text about? Use exactly one word."
)
price_schema = ResponseSchema(
    name="price",
    description="How expensive was the product? Use None if no price was provided in the text",
)

response_schemas = [sentiment_schema, subject_schema, price_schema]

In [9]:
parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"sentiment": string  // Is the text positive, neutral or negative? Only provide these words\n\t"subject": string  // What subject is the text about? Use exactly one word.\n\t"price": string  // How expensive was the product? Use None if no price was provided in the text\n}\n```'

Now we will make a more advanced template to instruct the model to generate a specific response structure, including JSON encoding.This is a key concept hence we can use it to generate any kind of response structure we want, including HTML, XML, or any other format. it can then be parsed to a different memory representation such as an object and be fed to another pipeline or process.

In [15]:
template = """
Interprete the text and evaluate the text.
sentiment: is the text in a positive, neutral or negative sentiment?
subject: What subject is the text about? Use exactly one word.

Just return the JSON, do not add ANYTHING, NO INTERPRETATION!

text: {input}

{format_instructions}

"""

from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(template=template)

format_instructions = parser.get_format_instructions()

message = prompt.format(
    input="I ordered Pizza Salami for 9.99$ and it was awesome!",
    format_instructions=format_instructions,
)
print(message)
prompt_template = ChatPromptTemplate.from_template(template=template)

response = llm(message)

response

Human: 
Interprete the text and evaluate the text.
sentiment: is the text in a positive, neutral or negative sentiment?
subject: What subject is the text about? Use exactly one word.

Just return the JSON, do not add ANYTHING, NO INTERPRETATION!

text: I ordered Pizza Salami for 9.99$ and it was awesome!

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"sentiment": string  // Is the text positive, neutral or negative? Only provide these words
	"subject": string  // What subject is the text about? Use exactly one word.
	"price": string  // How expensive was the product? Use None if no price was provided in the text
}
```




'```\n{\n    "sentiment": "positive",\n    "subject": "Pizza",\n    "price": "9.99$"\n}\n```'

In [17]:
output_dict = parser.parse(response)
output_dict

{'sentiment': 'positive', 'subject': 'Pizza', 'price': '9.99$'}